# Imports

In [9]:
import numpy as np # Pour les math
import matplotlib.pyplot as plt # Pour les graphiques
from scipy.optimize import linprog # Notre solveur

# Données

In [10]:
data = np.load("Temperatures-Montreal.npy")
print(data)

[ -0.83  -0.83  -0.83 ... -11.1  -11.1  -11.1 ]


# Résolution du système
## Variables
Soit ${e}_i$ la consomation électrique de la pompe à l'intervale $i$. \
Soit $s_i = 1$ si l'intervale $i$ se trouve le soir (entre 22H et 7H) et $s_i = 0$ sinon. \
Soit $j_i = 1$ si l'intervale $i$ se trouve la journée (entre 7H et 22H) et $j_i =0$ sinon. \
Soit $a_i = 1$ si on active la pompe à chaleur à l'intervale $i$ et $a_i = 0$ sinon. \
Soit $t_{{in}_i}$ la température intérieure à l'intervale $i$. \
Soit $t_{{ex}_i}$ la température exterieure à l'intervale $i$.

## Objectif
Minimiser le coût total de l'électricité consommée par la pompe à chaleur. \
$\[ \sum_{i=0}^{672} a_i*() \]$

## Contraintes
Consommation électrique maximale de la pompe à chaleur: $1kW$\
${e}_i \leq 1$ \
l'intervale ne peut pas être dans la journée et dans le soir. \
$s_i = 1 - j_i$

Les variables $ s_{i}$, $j_{i}$, $a_{i}$ sont binaires \
$s_i \in \{0, 1\}$ \
$j_i \in \{0, 1\}$ \
$a_i \in \{0, 1\}$

$
\begin{cases}
min a \\
s_i = 1 - j_i \\
{e}_i \leq 1 \\
\end{cases}
$

In [14]:
def task_1(i_idx):
    days_offset = 96
    end_idx = i_idx + days_offset*7
    central_value = np.mean(data[i_idx:end_idx])
    # On suppose qu'on commence la journée à 0
    print(central_value)

In [12]:
task_1(0)

-6.727605482178549
